In [1]:
from langchain.chat_models import init_chat_model
import os

model = init_chat_model(
    "qwen3-coder-plus",
    model_provider="openai",
    temperature=0,
    # max_tokens=500,
)
SYSTEM_PROMPT = """You are a system that generates Snowflake SQL queries based on user requests. Output only the SQL statement, with no extra text, explanations, or formatting. You only need to output the SQL statement, without any other markdown syntax decorations.
"""
from langchain.agents import create_agent

agent = create_agent(
    model=model,
    # system_prompt=SYSTEM_PROMPT,
)

response = agent.invoke({"messages": [{"role": "user", "content": "hi"}]})
for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

hi
================================== Ai Message ==================================

Hello! How can I assist you today?


In [ ]:
import pandas as pd
import snowflake.connector

df = pd.read_json("Spider2-main/spider2-snow/spider2-snow.jsonl", lines=True)

for index, row in df.iloc[23:24].iterrows():
    
    instance_id = row['instance_id']
    instruction = row['instruction']
    db_id = row['db_id']
    external_knowledge = row['external_knowledge']

    response = agent.invoke({"messages": [{"role": "user", "content": instruction}]})
    # for msg in response["messages"]:
    #     msg.pretty_print()

    sql_query = response["messages"][-1].content
    if "```sql" in sql_query:
        sql_query = sql_query.split("```sql")[1].split("```")[0].strip()

    conn = snowflake.connector.connect(
        user="ASHLII",
        password=os.getenv("SF_PD"),
        account="RSRSBDK-YDB67606",
        database=db_id)
    cur = conn.cursor()

    try:
        cur.execute(sql_query)
        rows = cur.fetchall()
        print(rows)
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        cur.close()
        conn.close()

An error occurred: 002003 (42S02): SQL compilation error:
Object 'PATENTS' does not exist or not authorized.
